# lab12 - st121413

## In-lab exercises

1. Spend some time to understand the Transformer and Speech Recognition examples.
 - I did this in the main_\[modelname\].py and save the best model in /checkpoint for the next one
 - https://github.com/akraradets/2021JanRTML/tree/main/lab12
2. After training, test the two models on some data (text, audio) you provide yourself

In [9]:
from myTransformer import *
from utils import *
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

vocab = load_vocab('checkpoint/transformer_vocab.pickle')
ntokens = len(vocab.stoi) # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
model.load_state_dict(torch.load('checkpoint/transformer.pth'))
model.to(device)

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=200, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=200, out_features=200, bias=True)
        (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=20

## Homework

Since you're all extremely busy with your projects (RIGHT??), for the homework for this week,
write an explanation of the steps that would be needed to adapt the PyTorch transformer to
the speech recognition task. Be specific in how you would do the implementation, but you do
not need to ac